In [ ]:
import scipy as sp
import numpy as np
import os
import glob
import sys
import pickle as pkl
from aopy import datareader, datafilter
import matplotlib.pyplot as plt
sys.path.append('/Users/mickey/aoLab/code/analyze/')
import tfspec

In [ ]:
data_path = '/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/'
data_file_list = glob.glob(os.path.join(data_path,'*','*','*.clfp.dat'))
print(len(data_file_list))

In [ ]:
# conservative downsampling method
def downsample_binary_mask(mask,dsf):
    mask_ds_len = int(np.round(len(mask)/dsf))
    sample_idx = np.arange(0,len(mask),dsf)
    mask_ds = np.zeros(mask_ds_len,dtype=bool)
    for ds_idx, idx in enumerate(sample_idx):
        ds_window = idx + np.arange(0,dsf)
        ds_window = ds_window[ds_window < len(mask)]
        mask_ds[ds_idx] = np.any(mask[ds_window])
    if mask[sample_idx[-1]]: # if the last checked sample is bad, assume the rest of it is also bad
        mask_ds[ds_idx+1:] = True
    return mask_ds

In [ ]:
def create_ds_file(file_name,srate_down,overwrite):
    save_dir = os.path.dirname(file_name)
    file_basename = os.path.basename(file_name)
    ds_file_name = os.path.join(save_dir,file_basename[:-4] + f'_ds{int(srate_down)}.dat')
    ds_mask_file_name = os.path.join(save_dir,file_basename[:-4] + f'_ds{int(srate_down)}.mask.pkl')
    # make filter ranges
    f_range_list = [[0,10],
                    [10,20],
                    [20,30],
                    [30,40],
                    [40,50],
                    [0,20],
                    [10,30],
                    [20,40],
                    [30,50],
                    [0,30],
                    [10,40],
                    [20,50]]
    if os.path.exists(ds_file_name) and not overwrite:
        print(f'{ds_file_name} already exists.')
    else:
        # load file
        print(file_name)
        data,exp,mask = datareader.load_ecog_clfp_data(file_name)
        n_ch = exp['num_ch']
        srate_in = exp['srate']
        # downsample data
        dsf = srate_in // srate_down
        mask_all = mask['hf'] | mask['sat']
        ds_n = int(np.ceil(data.shape[-1]/dsf))
        data_down = np.zeros((n_ch,ds_n),dtype=np.float32)
        mask_ds_hf = np.zeros((ds_n),dtype=bool)
        ds_n_idx = len(mask_ds_hf)
        mask_ds_hf[:ds_n_idx] = mask['hf'][::dsf]
        mask_ds_sat = np.zeros((ds_n),dtype=bool)
        mask_ds_sat = mask['sat'][::dsf]
        if ds_n > ds_n_idx:
            mask_ds_hf[-1] = mask_ds_hf[-2]
            mask_ds_sat[-1] = mask_ds_sat[-2]
        mask_ds = {'hf': mask_ds_hf,
                   'sat': mask_ds_sat}
        mask_ds_all = mask_ds['hf'] | mask_ds['sat']
        print(f'{100*mask_ds_all.mean()}% of data masked')
        print('downsampling data...')
        print_progress_bar(0,n_ch)
        for ch_idx in range(n_ch):
            data_down[ch_idx,:] = np.float32(sp.signal.decimate(data[ch_idx,:],dsf))
            print_progress_bar(ch_idx,n_ch)
        data = data_down.copy()
        del data_down
        data[:,mask_ds_all] = 0.
        # save file!
        print(f'saving data to: {ds_file_name}')
        data.tofile(ds_file_name)
        print(f'saving mask to: {ds_mask_file_name}')
        with open(ds_mask_file_name,'wb') as f:
            pkl.dump(mask_ds,f)
        # filter data
        for f_range in f_range_list:
            ds_filt_filename = file_basename[:-4] + f'_ds{int(srate_down)}_fl{f_range[0]}u{f_range[1]}.dat'
            if os.path.exists(os.path.join(save_dir,ds_filt_filename)):
                print(f'{os.path.join(save_dir,ds_filt_filename)} already exists...')
            else:
                print(f'Filtering data to {f_range}Hz')
                n_tap = srate_down
                a_fir = 1
                pass_zero = f_range[0] == 0
                if pass_zero:
                    f_use = f_range[1]
                else:
                    f_use = f_range
                b_fir = sp.signal.firwin(n_tap,f_use,fs=srate_down,pass_zero=pass_zero)
                data_filt = np.float32(sp.signal.filtfilt(b_fir,a_fir,data,axis=-1))
                # save data
                print(f'Saving filtered data to {os.path.join(save_dir,ds_filt_filename)}')
                data_filt.tofile(os.path.join(save_dir,ds_filt_filename))

In [ ]:
# simple progressbar, not tied to the iterator
def print_progress_bar(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [ ]:
srate_down = 250
overwrite = True
for ecog_file in data_file_list:
    create_ds_file(ecog_file,srate_down,overwrite,)

In [ ]:
# renaming those files: the '*.clfp.ds___.dat' doesn't play so well with the datareader.
# this will not run if the poorly named files no longer exist - no need to remove.
data_ds_file_list = glob.glob('/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/*/*/*.ds250.dat')
for data_file in data_ds_file_list:
    file_dir_name = os.path.dirname(data_file)
    file_parts = os.path.basename(data_file).split('.')
    new_data_basename = f'{file_parts[0]}.{file_parts[1]}.clfp_ds250.{file_parts[-1]}'
    new_data_file = os.path.join(file_dir_name,new_data_basename)
    print(f'old file: {data_file}')
    print(f'new file: {new_data_file}')
    os.system(f'mv {data_file} {new_data_file}')

In [ ]:
# test ds data files! Try loading one and plotting a bit of it.
data_test,exp,_ = datareader.load_ecog_clfp_data('/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/180328/006/rec006.LM1_ECOG_3.clfp_ds250.dat')

In [ ]:
print(data_test.shape)
f,ax = plt.subplots(1,1,figsize=(16,8))
t_offset = 10
plot_idx = np.arange(1000) + t_offset*250
ax.plot(plot_idx/250,data_test[:,plot_idx].T+1000.*np.arange(exp['num_ch']));

In [ ]:
# :) now let's look at a single channel
plt.plot(data_test[30,np.arange(500,1500)])

Looks pretty good to me! How about a PSD?

In [ ]:
window_t = 4
bw = 2
nw = window_t*bw/2
n_taper = 2*nw - 1
tapers = [window_t, nw, n_taper]
sgram,f_sgram,ti_sgram,_ = tfspec.tfspec(data_test[30:31,:],sampling=250,tapers=tapers,dn=0.5)
sgram = np.float32(sgram) # save some space!

In [ ]:
f,ax = plt.subplots(1,1,figsize=(20,6))
extent = (ti_sgram[0]/250, ti_sgram[-1]/250, f_sgram[0], f_sgram[-1])
sgram_img = ax.imshow(10*np.log10(sgram[0,]).T,clim=(30,60),extent=extent,origin='bottom',aspect='auto')
plt.colorbar(sgram_img)

Looks reasonable to me! I'll still probably run data quality checks and masks when loading these downsamples files, but this looks OK to me!